In [1]:
# Libraries to manage data
import numpy as np
import pandas as pd

# Libraries for reqs and scrapping
import requests
from bs4 import BeautifulSoup

In [2]:
# Raw html response
covid_url = 'https://www.worldometers.info/coronavirus/'
req = requests.get(covid_url)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# From table get what we want
# html tags: 
# variables names --> 'thead' --> 'tr' --> 'th'
# variables names --> 'tbody' --> 'tr' --> 'td'
variables = []
variables_html = soup.find_all('thead')[-1].find_all('tr')
for tag in variables_html:
    ths = tag.find_all(['th'])
    row = [th.text for th in ths]
    variables.append(row)
# variables

samples = []
samples_html = soup.find_all('tbody')[0].find_all('tr')
for tr in samples_html:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    samples.append(row)
# samples

df = pd.DataFrame(samples[:len(samples)-6], columns=variables[0]).drop(columns=["#"])       
df.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,\nNorth America\n,"20,352,667","+57,205","467,481","+1,486","12,343,529","+20,936","7,541,657","33,726",,,,,,North America,\n,,
1,\nAsia\n,"19,594,118","+102,154","319,475","+1,329","17,828,448","+107,048","1,446,195","27,214",,,,,,Asia,\n,,
2,\nSouth America\n,"12,280,038","+11,430","345,686",+162,"10,858,312","+1,294","1,076,040","16,845",,,,,,South America,\n,,
3,\nEurope\n,"20,884,971","+176,203","482,759","+3,688","9,937,237","+111,205","10,464,975","26,564",,,,,,Europe,\n,,
4,\nAfrica\n,"2,470,482","+6,297","58,015",+105,"2,087,551","+6,048","324,916","2,759",,,,,,Africa,\n,,


In [4]:
df = df.iloc[8:,:-3].reset_index(drop=True)

In [5]:
# rename columns
df.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']
df = df[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]
df.sample(5)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
140,Djibouti,Africa,"994,610","5,780",+10,61,,"5,641",+6,78,,"5,811",61,"97,035","97,561"
141,Mayotte,Africa,"275,817","5,708",+92,53,,"2,964",,"2,691",4,"20,695",192,"34,115","123,687"
171,Mongolia,Asia,"3,302,544",941,+18,,,473,+89,468,4,285,,"471,863","142,879"
62,Oman,Asia,"5,165,792","127,019",,"1,483",,"119,009",,"6,527",43,"24,588",287,"571,472","110,626"
208,Greenland,North America,"56,816",19,,,,18,,1,,334,,"14,482","254,893"


In [17]:
# fix data
pd.set_option('display.max_rows', 500)


for col in df.columns[2:]:
    # replace comma with empty string
    df[col] = df[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df[col] = df[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df = df.replace('', np.nan)

# rename here what ever we want
df = df.rename(columns = {
    "Country/Region":"Country"
})
df = df[['Country','Population','TotalCases','TotalDeaths']]


df["Country"].loc[df["Country"]=="USA"]="United States"
df["Country"].loc[df["Country"]=="UK"]="United Kingdom"
df["Country"].loc[df["Country"]=="Taiwan"]="Taiwan Province of China"
df["Country"].loc[df["Country"]=="Congo"]="Congo (Kinshasa)"
df["Country"].loc[df["Country"]=="Czechia"]="Czech Repiublic"
df["Country"].loc[df["Country"]=="S. Korea"]="South Korea"
df["Country"].loc[df["Country"]=="Palestine"]="Palestinian Territories"



df

,Country,Population,TotalCases,TotalDeaths
0,United States,331902144,17672842,318672
1,India,1386271357,10004620,145167
2,Brazil,213258918,7120103,184992
3,Russia,145963586,2791220,49762
4,France,65340640,2427316,59619
5,Turkey,84757932,1982090,17610
6,United Kindom,68051351,1977167,66541
7,Italy,60420388,1906377,67220
8,Spain,46763207,1805633,48777
9,Argentina,45387444,1524372,41534


In [15]:
# save as .csv file
df.to_csv('covid19_data.csv', index=False)